<div class="alert alert-danger" role="alert">
    <span style="font-size:20px">&#9888;</span> <span style="font-size:16px">This is a read-only notebook! If you want to make and save changes, save a copy by clicking on <b>File</b> &#8594; <b>Save a copy</b>. If this is already a copy, you can delete this cell.</span>
</div>

# Datatype transformations

This notebook provides methods to ensure all the variables your dataset are represented by the correct datatype.

<a id="table-of-contents"></a>
**Table of contents**

<ul class="toc-item"><li><span><a href="#Datatype-transformations" data-toc-modified-id="Datatype-transformations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Datatype transformations</a></span></li><li><span><a href="#Quick-dataset-overview" data-toc-modified-id="Quick-dataset-overview-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Quick dataset overview</a></span></li><li><span><a href="#Converting-dataframe-columns-to-a-different-datatype" data-toc-modified-id="Converting-dataframe-columns-to-a-different-datatype-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Converting dataframe columns to a different datatype</a></span><ul class="toc-item"><li><span><a href="#Converting-to-a-numeric-datatype" data-toc-modified-id="Converting-to-a-numeric-datatype-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Converting to a numeric datatype</a></span></li><li><span><a href="#Datetime-conversions" data-toc-modified-id="Datetime-conversions-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Datetime conversions</a></span><ul class="toc-item"><li><span><a href="#Converting-integers-to-dates" data-toc-modified-id="Converting-integers-to-dates-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Converting integers to dates</a></span></li><li><span><a href="#Extract-date-values-from-complex-strings" data-toc-modified-id="Extract-date-values-from-complex-strings-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Extract date values from complex strings</a></span></li><li><span><a href="#Extract-date-value-from-an-Excel-date-read-in-as-number" data-toc-modified-id="Extract-date-value-from-an-Excel-date-read-in-as-number-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Extract date value from an Excel date read in as number</a></span></li><li><span><a href="#Converting-arrays,-lists,-or-pandas-Series" data-toc-modified-id="Converting-arrays,-lists,-or-pandas-Series-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Converting arrays, lists, or pandas Series</a></span></li><li><span><a href="#Using-multiple-columns-in-a-pandas-DataFrame-to-construct-the-date" data-toc-modified-id="Using-multiple-columns-in-a-pandas-DataFrame-to-construct-the-date-3.2.5"><span class="toc-item-num">3.2.5&nbsp;&nbsp;</span>Using multiple columns in a pandas DataFrame to construct the date</a></span></li></ul></li><li><span><a href="#Date-related-calculations" data-toc-modified-id="Date-related-calculations-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Date-related calculations</a></span></li><li><span><a href="#Date-aggregation-/-splitting" data-toc-modified-id="Date-aggregation-/-splitting-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Date aggregation / splitting</a></span><ul class="toc-item"><li><span><a href="#Split-to-less-granular-dates" data-toc-modified-id="Split-to-less-granular-dates-3.4.1"><span class="toc-item-num">3.4.1&nbsp;&nbsp;</span>Split to less granular dates</a></span></li><li><span><a href="#Aggregate-to-more-granular-dates" data-toc-modified-id="Aggregate-to-more-granular-dates-3.4.2"><span class="toc-item-num">3.4.2&nbsp;&nbsp;</span>Aggregate to more granular dates</a></span></li></ul></li></ul></li></ul>

This notebook primarily uses capabilities from numpy and pandas. 

**We begin by importing key libraries**

In [1]:
# Import key libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser as dparser
from datetime import datetime

**Optional import of OW color scheme**

In [2]:
# Load in OW color scheme and plot style
plt.style.use('../../utilities/resources/ow_style.mplstyle')

# Add path of the folder 'utilities' to the path from which we can import modules  
import sys
sys.path.append('../../utilities')
from resources.ow_colormap import ow_colormap 

**Load in data from CSV**

We read in the data from a CSV containing data about used car auction sales

In [3]:
dataset = pd.read_csv("sample_input/transformations_used_cars.csv", low_memory=False)

<a id="dataset_overview"></a>
# Quick dataset overview

We use the following pandas methods to obtain basic information about the contents of the data:
* .info(): Column names, number of non-nulls, and column data type
* .head(): See top rows of each data field
* .describe(): Basic stats for each variable, which we then augment with number of missing or blank values

Following this we use a custom function for additional information from numerical and categorical variables respectively.

In [4]:
dataset.head(2)

,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,Transmission,WheelTypeID,WheelType,VehOdo,Nationality,Size,MMRAcquisitionAuctionAveragePrice,VehBCost,WarrantyCost
0,0,6/17/2009,MANHEIM,2001,8.0,NISSAN,ALTIMA 2.4L I4 EFI,GXE,4D SEDAN GXE,WHITE,AUTO,2.0,Covers,80702.0,TOP LINE ASIAN,MEDIUM,2942.0,4160.0,1023
1,0,10/5/2010,OTHER,2008,Five,FORD,TAURUS,SEL,4D SEDAN SEL,SILVER,AUTO,1.0,Alloy,88245.0,AMERICAN,MEDIUM,9817.0,7850.0,1633


In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   IsBadBuy                           5000 non-null   int64  
 1   PurchDate                          5000 non-null   object 
 2   Auction                            5000 non-null   object 
 3   VehYear                            5000 non-null   int64  
 4   VehicleAge                         4783 non-null   object 
 5   Make                               5000 non-null   object 
 6   Model                              5000 non-null   object 
 7   Trim                               4846 non-null   object 
 8   SubModel                           5000 non-null   object 
 9   Color                              5000 non-null   object 
 10  Transmission                       5000 non-null   object 
 11  WheelTypeID                        3603 non-null   float

<a id="detailed_dataset_overview"></a>
# Converting dataframe columns to a different datatype 

The seven common datatypes encountered when using Pandas are:

| Datatype  |  Usage                                        |
|:----------|:---------------------------------------------:|
| object    |  Text or mixed numeric and non-numeric values | 
| float     |  Floating point numbers                       |   
| int       |  Integer numbers                              | 
| bool      |  True/False values                            | 
| datetime  |  Date and time values                         |   
| timedelta |  Difference between two datetime values       |
| category  |  Finite list of text values                   |

Pandas does not always infer the appropriate datatype, if there are inconsistent types in a single column. There are a few functions which allows us to convert a column to a different datatype, demonstrated below.

-------------------------------------------------------------------------------------------------------------------------------
## Converting to a numeric datatype

**pd.to_numeric will attempt to convert a column to an integer or float type**

It will raise an error if any of the values cannot be converted to a numeric type. You can instead pass the `errors='coerce'`, which will convert any non-numeric values to NaN:

In [6]:
dataset['VehicleAge'] = pd.to_numeric(dataset['VehicleAge'], errors='coerce')
dataset['VehicleAge']

0       8.0
1       NaN
2       8.0
3       4.0
4       4.0
       ... 
4995    6.0
4996    NaN
4997    4.0
4998    3.0
4999    7.0
Name: VehicleAge, Length: 5000, dtype: float64

**Using the `downcast` option allows you to choose whether the data will be cast as integer or float. Additionally, it will automatically use the smallest integer/float possible to save memory**

In [7]:
dataset['VehicleAge'] = pd.to_numeric(dataset['VehicleAge'], errors='coerce', downcast='float')
dataset['VehicleAge']

0       8.0
1       NaN
2       8.0
3       4.0
4       4.0
       ... 
4995    6.0
4996    NaN
4997    4.0
4998    3.0
4999    7.0
Name: VehicleAge, Length: 5000, dtype: float32

-------------------------------------------------------------------------------------------------------------------------------
**The .astype() method allows you to convert a column to any recognised Pandas datatype**

In [8]:
dataset['WheelType'] = dataset['WheelType'].astype('category')
dataset['WheelType'].dtype

CategoricalDtype(categories=['Alloy', 'Covers', 'Special'], ordered=False)

You can pass a dictionary argument to convert multiple columns at once:

In [9]:
dataset = dataset.astype({'Auction': 'category', 'WarrantyCost': 'float32'})
dataset[['Auction', 'WarrantyCost']].dtypes

Auction         category
WarrantyCost     float32
dtype: object

**As with `to_numeric`, the conversion will fail if the column contains values that cannot be converted** 

In [10]:
import sys
sys.path.append('../../utilities')
from data_exploration import dataset_overview

<a id="numerical_variable_check"></a>
## Datetime conversions

The pandas `to_datetime` functionality is good at identifying the type of string passed through and converting into datetime using a variety of format recognition patterns:

In [11]:
numeric_string_date = pd.to_datetime('20221231')
typed_out_date = pd.to_datetime('December 31 2022')
with_dashes_date = pd.to_datetime('12-31-2022')
with_slashes_date = pd.to_datetime('12/31/2022')
abbreviated_date = pd.to_datetime('Dec 31 2022')

print(numeric_string_date, typed_out_date, with_dashes_date, 
      with_slashes_date, abbreviated_date, sep='\n')

2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00


When you know the expected type of format, it is still preferred to explicitly state the format. It will warn you when the format doesn’t match, and is faster:

In [12]:
pd.to_datetime('12/31/2022', format = '%m/%d/%Y')

Timestamp('2022-12-31 00:00:00')

A good cheatsheet can be found here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior.

<a id="numerical_variable_correlations"></a>
### Converting integers to dates

To convert an integer to a date, specify the format in which the date integer is provided in.

In [13]:
# Integer typed out
integer1 = 20221231
integer2 = 12312022
integer3 = 31122022

numeric_date1 = pd.to_datetime(integer1, format='%Y%m%d')
numeric_date2 = pd.to_datetime(integer2, format='%m%d%Y')
numeric_date3 = pd.to_datetime(integer3, format='%d%m%Y')

print(numeric_date1, numeric_date2, numeric_date3, sep='\n')

2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00


<a id="non_numerical_variable_check"></a>
### Extract date values from complex strings

Using the dateutil parsing function, you can find string patterns that are dates from a larger string, though you want to be careful in doing this since it can find unrelated numbers and think it is a date

In [14]:
string1 = dparser.parse("oliver december 31 digital 2022 wyman", fuzzy=True)
string2 = dparser.parse("oliver 12 31 digital 2022 wyman", fuzzy=True)
string3 = dparser.parse("2022-12-31", fuzzy=True)
string4 = dparser.parse("ow 31 wyman oliver 12 digital 2022", fuzzy=True)

print(string1, string2, string3, string4, sep='\n')

2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00
2022-12-31 00:00:00


In most cases you will be better off identifying a pattern in your strings, and use substrings to piece together the date

In [15]:
string1 = "Year 2019, on June 3rd"
string2 = "Year 1990, on May 31st"
string3 = "Year 2001, on December 5th"
print(pd.to_datetime(string1[5:9] + " " + string1[14:-2], format="%Y %B %d"))
print(pd.to_datetime(string2[5:9] + " " + string2[14:-2], format="%Y %B %d"))
print(pd.to_datetime(string3[5:9] + " " + string3[14:-2], format="%Y %B %d"))

2019-06-03 00:00:00
1990-05-31 00:00:00
2001-12-05 00:00:00


### Extract date value from an Excel date read in as number

See the answer at https://stackoverflow.com/questions/31359150/convert-date-from-excel-in-number-format-to-date-format-python/31359287 for more information. Here is a simple example.

In [16]:
# For integer dates after 1900/03/01:
excel_date = 44769
pd.to_datetime(datetime.fromordinal(datetime(1900, 1, 1).toordinal() + excel_date - 2))

Timestamp('2022-07-27 00:00:00')

### Converting arrays, lists, or pandas Series

A benefit of `pd.to_datetime` is that it works with all of these directly, so you don’t have to write loops / list comprehensions / pandas apply functions:

In [17]:
pd.to_datetime(np.array(['2018/12/31', '2013/02/28', '2019/05/01', '2017/11/11']), format="%Y/%m/%d")

DatetimeIndex(['2018-12-31', '2013-02-28', '2019-05-01', '2017-11-11'], dtype='datetime64[ns]', freq=None)

In [18]:
pd.to_datetime(['2018/12/31', '2013/02/28', '2019/05/01', '2017/11/11'], format="%Y/%m/%d")

DatetimeIndex(['2018-12-31', '2013-02-28', '2019-05-01', '2017-11-11'], dtype='datetime64[ns]', freq=None)

In [19]:
df = pd.DataFrame({"DateString": ['2018/12/31', '2013/02/28', '2019/05/01', '2017/11/11'],
                          "Value": [1,2,3,4]})

df['Date'] = pd.to_datetime(df["DateString"], format="%Y/%m/%d")
display(df)
df.dtypes

,DateString,Value,Date
0,2018/12/31,1,2018-12-31
1,2013/02/28,2,2013-02-28
2,2019/05/01,3,2019-05-01
3,2017/11/11,4,2017-11-11


DateString            object
Value                  int64
Date          datetime64[ns]
dtype: object

### Using multiple columns in a pandas DataFrame to construct the date

The easiest approach is to use the `apply` function:

In [20]:
df = pd.DataFrame({"Year": [1990, 2010, 2011, 2004], "Quarter": [2,1,3,4]})

df['First_Month'] = df.apply(lambda x: datetime(
    year = x["Year"], month = 3*(x["Quarter"]-1) + 1, day = 1), axis=1)
df['Last_Month'] = df.apply(lambda x: datetime(
    year = x["Year"], month = 3*x["Quarter"], day = 1), axis=1)
df

,Year,Quarter,First_Month,Last_Month
0,1990,2,1990-04-01,1990-06-01
1,2010,1,2010-01-01,2010-03-01
2,2011,3,2011-07-01,2011-09-01
3,2004,4,2004-10-01,2004-12-01


## Date-related calculations

You can add and subtract two different dates easily (to get days, months, years, etc.):

In [21]:
from_date = pd.to_datetime('20180204')
to_date = pd.to_datetime('20220130')

print(to_date - from_date)
print((to_date - from_date).days / 365.24)

1456 days 00:00:00
3.9864198882926294


You can also add and subtract periods to a date using Pandas' `Timedelta` datatype:

In [22]:
print(from_date + pd.Timedelta(days=365))
print(from_date - pd.Timedelta(weeks=4))

2019-02-04 00:00:00
2018-01-07 00:00:00


The `bdate_range` function allows you to count the number of non-weekend days in a given period. *Note - this does not take into account bank holidays.*

In [23]:
pd_datetime1 = pd.to_datetime('20211231')
pd_datetime2 = pd.to_datetime('20221231')

pd_datetime_count = len(pd.bdate_range(pd_datetime1, pd_datetime2))
print("Number of business days in 2022:", pd_datetime_count)

Number of business days in 2022: 261


## Date aggregation / splitting

We can aggregate dates, e.g. transform monthly to quarterly. As well as split time, e.g. get weekly data from monthly data. In order to do this, the index for your time series needs to be a Pandas `datetime` type:

In [24]:
ts_data = pd.read_csv('sample_input/ts_data.csv')
ts_data = ts_data.assign(datetime = pd.to_datetime(ts_data['date'])).set_index("datetime")
ts_data.head()

,date,sales
datetime,,
2013-01-31,2013-01-31,3794
2013-02-28,2013-02-28,3863
2013-03-31,2013-03-31,5190
2013-04-30,2013-04-30,5783
2013-05-31,2013-05-31,6298


###  Split to less granular dates

We can aggregrate our data to a different frequency using the `resample` method:

In [25]:
# Create quarterly data based on mean of monthly sales
ts_data.resample('Q').mean().head()

,sales
datetime,
2013-03-31,4282.333333
2013-06-30,6273.333333
2013-09-30,6430.666667
2013-12-31,5254.666667
2014-03-31,4858.000000


### Aggregate to more granular dates

In [26]:
# Create weekly data from monthly. The monthly value is assigned to the first week
ts_data.resample('W').mean().head(10)

,sales
datetime,
2013-02-03,3794.0
2013-02-10,NaN
2013-02-17,NaN
2013-02-24,NaN
2013-03-03,3863.0
2013-03-10,NaN
2013-03-17,NaN
2013-03-24,NaN
2013-03-31,5190.0


In [27]:
# Interpolation
ts_data.resample('W').mean().interpolate(method='time').head()

,sales
datetime,
2013-02-03,3794.00
2013-02-10,3811.25
2013-02-17,3828.50
2013-02-24,3845.75
2013-03-03,3863.00


[Table of contents](#table-of-contents)